## Name : Diksha Pardeshi
## USC ID : 7961073499
## GitHub username : dikshapardeshi


#### 1. Time Series Classication Part 1: Feature Creation/Extraction

While going through the data, I noticed that the dataset4.csv file in the bending2 folder was not in the standard CSV format. I corrected this by replacing spaces with commas using Google Sheets. Additionally, I found extra commas in the dataset9.csv and dataset14.csv files in the cycling folder, which I removed manually using Sublime Text Editor.

#### Download the AReM data


In [1]:
import numpy as np
import os
import pandas as pd
from scipy.stats import bootstrap

import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

data_file_path = '../data'
folders = [folder for folder in os.listdir(data_file_path) if folder != '.DS_Store']

print(folders)


['bending1', 'walking', 'bending2', 'standing', 'sitting', 'lying', 'cycling']


All the data is present in the data folder

#### (b) Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1,
2, and 3 in other folders as test data and other datasets as train data.

In [3]:
def loading_test_train_data(data_file_path):
    
    train_df = pd.DataFrame()  
    test_df = pd.DataFrame()   

    #folders = ['bending1', 'bending2', 'cycling', 'lying', 'sitting', 'standing', 'walking']

    for folder in folders:
        folder_path = os.path.join(data_file_path, folder)

        dataset_files = []

        for file in sorted(os.listdir(folder_path)):
            if file.endswith('.csv'):
                dataset_files.append(file)

        for file in dataset_files:
            data = pd.read_csv(os.path.join(folder_path, file), skiprows=4, usecols=range(7))

            if folder in ['bending1', 'bending2'] and file in ['dataset1.csv', 'dataset2.csv']:
                test_df = pd.concat([test_df, data], ignore_index=True)
            elif folder not in ['bending1', 'bending2'] and file in ['dataset1.csv', 'dataset2.csv', 'dataset3.csv']:
                test_df = pd.concat([test_df, data], ignore_index=True)
            else:
                train_df = pd.concat([train_df, data], ignore_index=True)

    return train_df,test_df

train_data, test_data = loading_test_train_data(data_file_path)



In [4]:
train_data.head()

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,42.00,0.71,21.25,0.43,30.00,0.00
1,250,41.50,0.50,20.25,1.48,31.25,1.09
2,500,41.50,0.50,14.25,1.92,33.00,0.00
3,750,40.75,0.83,15.75,0.43,33.00,0.00
4,1000,40.00,0.71,20.00,2.74,32.75,0.43


In [5]:
test_data.head()

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,39.25,0.43,22.75,0.43,33.75,1.3
1,250,39.25,0.43,23.00,0.00,33.00,0.0
2,500,39.25,0.43,23.25,0.43,33.00,0.0
3,750,39.50,0.50,23.00,0.71,33.00,0.0
4,1000,39.50,0.50,24.00,0.00,33.00,0.0


####  (c) Feature Extraction

Classifcation of time series usually needs extracting features from them. In this
problem, we focus on time-domain features.

i. Research what types of time-domain features are usually used in time series
classication and list them (examples are minimum, maximum, mean, etc).

1. Mean: The average value of the entire time series.
2. Minimum: The smallest value in the entire time series.
3. Maximum: The largest value in the entire time series.
4. Range: The difference between the maximum and minimum values.
5. Median: The middle value when the series is ordered.
6. Standard Deviation: Measures the spread/variability of the time series around the mean.
7. Variance: Square of the standard deviation
8. Skewness: Describes the asymmetry of the distribution of values.
9. Kurtosis: Measures the "tailedness" of the distribution; a high kurtosis indicates heavy tails.
10. Sum: The total sum of all values in the time series.
11. Slope: The overall trend or linear slope of the time series.
12. Number of Peaks: Count of significant peaks (local maxima) in the time series.
13. First Quartile (Q1): This is the value below which 25% of the data points in the time series fall.
14. Third Quartile (Q3): This is the value below which 75% of the data points in the time series fall. 
15. Interquartile Range (IQR): The IQR is the difference between the third quartile (Q3) and the first quartile (Q1) and represents the middle 50% of the data. 

ii. Extract the time-domain features minimum, maximum, mean, median, standard
deviation,  first quartile, and third quartile for all of the 6 time series
in each instance.

In [6]:
def load_data(data_file_path):
    data_list = []  
    for folder in folders:
        folder_path = os.path.join(data_file_path, folder)

        for file in sorted(os.listdir(folder_path)):
            if file.endswith('.csv'):
                data = pd.read_csv(os.path.join(folder_path, file), skiprows=4, usecols=range(7))
                data_list.append(data)

    return data_list

In [7]:
def calculate_stats(data_list):
    result_data = []  
    for i, data in enumerate(data_list):
        
        
        stats = data.describe().T  
        
        
        instance_stats = {'Instance': i + 1}  
        
        for col in stats.index[1:]:
            instance_stats[f'min_{col}'] = stats.at[col, 'min']
            instance_stats[f'max_{col}'] = stats.at[col, 'max']
            instance_stats[f'mean_{col}'] = stats.at[col, 'mean']
            instance_stats[f'median_{col}'] = stats.at[col, '50%']  
            instance_stats[f'std_dev_{col}'] = stats.at[col, 'std']
            instance_stats[f'1st_quart_{col}'] = stats.at[col, '25%']  
            instance_stats[f'3rd_quart_{col}'] = stats.at[col, '75%']  

        result_data.append(instance_stats)

   
    result_df = pd.DataFrame(result_data)
    
    return result_df


In [8]:

data_list = load_data(data_file_path)

result_df = calculate_stats(data_list)


In [9]:
result_df

,Instance,min_avg_rss12,max_avg_rss12,mean_avg_rss12,median_avg_rss12,std_dev_avg_rss12,1st_quart_avg_rss12,3rd_quart_avg_rss12,min_var_rss12,max_var_rss12,...,std_dev_avg_rss23,1st_quart_avg_rss23,3rd_quart_avg_rss23,min_var_rss23,max_var_rss23,mean_var_rss23,median_var_rss23,std_dev_var_rss23,1st_quart_var_rss23,3rd_quart_var_rss23
0,1,37.25,45.00,40.624792,40.50,1.476967,39.25,42.0000,0.0,1.30,...,2.188449,33.0000,36.00,0.0,1.92,0.570583,0.430,0.582915,0.0000,1.3000
1,2,38.00,45.67,42.812812,42.50,1.435550,42.00,43.6700,0.0,1.22,...,1.995255,32.0000,34.50,0.0,3.11,0.571083,0.430,0.601010,0.0000,1.3000
2,3,35.00,47.40,43.954500,44.33,1.558835,43.00,45.0000,0.0,1.70,...,1.999604,35.3625,36.50,0.0,1.79,0.493292,0.430,0.513506,0.0000,0.9400
3,4,33.00,47.75,42.179812,43.50,3.670666,39.15,45.0000,0.0,3.00,...,3.849448,30.4575,36.33,0.0,2.18,0.613521,0.500,0.524317,0.0000,1.0000
4,5,33.00,45.75,41.678063,41.75,2.243490,41.33,42.7500,0.0,2.83,...,2.411026,28.4575,31.25,0.0,1.79,0.383292,0.430,0.389164,0.0000,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,84,26.50,44.33,36.687292,36.00,3.529404,34.25,39.3725,0.0,12.89,...,2.978238,14.6700,18.50,0.0,8.19,3.073313,2.690,1.629675,1.9125,4.0875
84,85,25.33,45.00,37.114313,36.25,3.710385,34.50,40.2500,0.0,10.84,...,2.847876,14.7500,18.50,0.0,9.50,3.076354,2.770,1.824534,1.7000,4.0375
85,86,26.75,44.75,36.863375,36.33,3.555787,34.50,39.7500,0.0,11.68,...,2.655906,15.0000,18.67,0.0,8.81,2.773313,2.590,1.569919,1.6400,3.6325
86,87,26.25,44.25,36.957458,36.29,3.434863,34.50,40.2500,0.0,8.64,...,2.851673,14.0000,18.25,0.0,8.34,2.934625,2.525,1.631380,1.6600,4.0300


iii. Estimate the standard deviation of each of the time-domain features you
extracted from the data. Then, use Python's bootstrapped or any other
method to build a 90% bootsrap con dence interval for the standard deviation
of each feature.

In [10]:
std_ = result_df.describe().loc[['std']].T
std_df =std_[1:]

std_df

,std
min_avg_rss12,9.569975
max_avg_rss12,4.394362
mean_avg_rss12,5.335718
median_avg_rss12,5.440054
std_dev_avg_rss12,1.772153
1st_quart_avg_rss12,6.153590
3rd_quart_avg_rss12,5.138925
min_var_rss12,0.000000
max_var_rss12,5.062729
mean_var_rss12,1.574164


In [11]:
confidence_intervals = []

for feature_name in std_df.index:
    std_value = std_df.loc[feature_name].values.flatten()
    
    data = (np.random.normal(loc=0, scale=std_value, size=1000),)      
    bootstrap_result = bootstrap(data, np.std, confidence_level=0.9, method='basic')

    lower_bound = bootstrap_result.confidence_interval.low
    upper_bound = bootstrap_result.confidence_interval.high
    confidence_intervals.append([feature_name, lower_bound, upper_bound])

df_CI = pd.DataFrame(confidence_intervals, columns=["Feature", "Lower Bound", "Upper Bound"])

display(df_CI)


,Feature,Lower Bound,Upper Bound
0,min_avg_rss12,9.224180,9.961282
1,max_avg_rss12,4.156662,4.469835
2,mean_avg_rss12,5.185957,5.573687
3,median_avg_rss12,5.300631,5.716568
4,std_dev_avg_rss12,1.732748,1.869071
5,1st_quart_avg_rss12,5.948763,6.418370
6,3rd_quart_avg_rss12,4.983000,5.368694
7,min_var_rss12,0.000000,0.000000
8,max_var_rss12,4.860335,5.234609
9,mean_var_rss12,1.480777,1.593157


iv. Select Features


From the above data, I think, the three most important time-domain features are: 

Mean: The mean of the data points provides insight into the central tendency, accounting for the presence of various outliers.

Median: Given the numerous outliers that may skew the data, the median serves as a valuable feature because it is less influenced by extreme values.

Standard Deviation: Standard deviation indicates how much the data points typically deviate from the mean, offering insights into the distribution of the underlying data.

2. ISLR 3.7.4

I collect a set of data (n = 100 observations) containing a single
predictor and a quantitative response. I then fit a linear regression
model to the data, as well as a separate cubic regression, i.e. Y =
β0 + β1X + β2X2 + β3X3 + ϵ.



#### (a) Suppose that the true relationship between X and Y is linear, i.e. Y = β0 + β1X + ϵ. Consider the training residual sum of squares (RSS) for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.




We can expect the RSS for the cubic regression to be less than the RSS for linear regression since a cubic regression model will be more flexible and would be able to fit to the training data better. It's flexibility will give us  a lower RSS value.

#### (b) Answer (a) using test rather than training RSS.


We can expect the Linear regression RSS value to be smaller that the cubic regression model. A cubic model might overfit to the training data and would not perform as well on the testing data

#### (c) Suppose that the true relationship between X and Y is not linear, but we don’t know how far it is from linear. Consider the training RSS for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.


We can expect the RSS for the cubic regression to be less than the RSS for linear regression since a cubic regression model will be more flexible and would be able to fit to the training data better. It's flexibility will give us  a lower RSS value inspite of the fact that the true relationship between X and Y is not linear. 

#### (d) Answer (c) using test rather than training RSS.

There is not enough information since we dont know how far away from a linear relationship it is. If it was closer to a linear relation then RSS for linear regression would be lower but if the true relationship is closer to a cubic relationship then RSS for cubic regression would be lower.
